In [1]:
import numpy as np
import pandas as pd
from glob import glob

In [2]:
election_type = "councilors" # councilors

output = []
for path in glob(f"2021/{election_type}/*.xlsx"):
    print(path)
    xlsx = pd.ExcelFile(path)

    if election_type == "councilors":
        for sheet in xlsx.sheet_names:
            frag = pd.read_excel(xlsx, sheet_name=sheet)
            output.append(frag)
    else:
        frag = pd.read_excel(xlsx, sheet_name="Votación", header=6)
        frag["Candidato"] = frag["Nombres"] + " " + frag["Primer apellido"] + " " + frag["Segundo apellido"]
        frag = frag.rename(columns={"Votos": "Votos TER"})
        output.append(frag)

df = pd.concat(output, ignore_index=True)
df.head()

2021/councilors\01_Resultados_Mesa_Concejales_TER.xlsx
2021/councilors\02_Resultados_Mesa_Concejales_TER.xlsx
2021/councilors\03_Resultados_Mesa_Concejales_TER.xlsx
2021/councilors\04_Resultados_Mesa_Concejales_TER.xlsx
2021/councilors\05_Resultados_Mesa_Concejales_TER.xlsx
2021/councilors\06_Resultados_Mesa_Concejales_TER.xlsx
2021/councilors\07_Resultados_Mesa_Concejales_TER.xlsx
2021/councilors\08_Resultados_Mesa_Concejales_TER.xlsx
2021/councilors\09_Resultados_Mesa_Concejales_TER.xlsx
2021/councilors\10_Resultados_Mesa_Concejales_TER.xlsx
2021/councilors\11_Resultados_Mesa_Concejales_TER.xlsx
2021/councilors\12_Resultados_Mesa_Concejales_TER.xlsx
2021/councilors\13_Resultados_Mesa_Concejales_TER.xlsx
2021/councilors\14_Resultados_Mesa_Concejales_TER.xlsx
2021/councilors\15_Resultados_Mesa_Concejales_TER.xlsx
2021/councilors\16_Resultados_Mesa_Concejales_TER.xlsx


,Región Nº,Región,Provincia,CS,Distrito,Comuna,Circunscripción,Mesa Nº,Tipo,Fusionadas,...,Sub Pacto,Partido,Candidato,Votos TER,N° Región,Nº Mesa,SubPacto,N° mesa,Subpacto,MesaNº
0,1.0,DE TARAPACA,IQUIQUE,2.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,1.0,M,1M,...,PARTIDO COMUNISTA E INDEPENDIENTES,PARTIDO COMUNISTA DE CHILE,KARL HEINZ HARDER LLANOS,6.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,DE TARAPACA,IQUIQUE,2.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,1.0,M,1M,...,PARTIDO COMUNISTA E INDEPENDIENTES,PARTIDO COMUNISTA DE CHILE,HUMBERTO SADY GALAZ ROJAS,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,DE TARAPACA,IQUIQUE,2.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,1.0,M,1M,...,PARTIDO FEDERACION REGIONALISTA VERDE SOCIAL E...,FEDERACION REGIONALISTA VERDE SOCIAL,GIGLIOLA ANGELICA INOSTROZA ESCARATE,4.0,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,DE TARAPACA,IQUIQUE,2.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,1.0,M,1M,...,PARTIDO FEDERACION REGIONALISTA VERDE SOCIAL E...,FEDERACION REGIONALISTA VERDE SOCIAL,ESTER YAZMIN CISTERNAS VERA,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,DE TARAPACA,IQUIQUE,2.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,1.0,M,1M,...,PARTIDO FEDERACION REGIONALISTA VERDE SOCIAL E...,FEDERACION REGIONALISTA VERDE SOCIAL,GISELLE DEL CARMEN MUÑOZ GONZALEZ,4.0,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
list(df)

['Región Nº',
 'Región',
 'Provincia',
 'CS',
 'Distrito',
 'Comuna',
 'Circunscripción',
 'Mesa Nº',
 'Tipo',
 'Fusionadas',
 'Lista',
 'Nº en Voto',
 'Pacto',
 'Sub Pacto',
 'Partido',
 'Candidato',
 'Votos TER',
 'N° Región',
 'Nº Mesa',
 'SubPacto',
 'N°  mesa',
 'Subpacto',
 'MesaNº']

In [4]:
df = df.rename(columns={"Circunscripción electoral": "Circunscripción", "Mesa": "Mesa Nº"})

In [5]:
list(df)

['Región Nº',
 'Región',
 'Provincia',
 'CS',
 'Distrito',
 'Comuna',
 'Circunscripción',
 'Mesa Nº',
 'Tipo',
 'Fusionadas',
 'Lista',
 'Nº en Voto',
 'Pacto',
 'Sub Pacto',
 'Partido',
 'Candidato',
 'Votos TER',
 'N° Región',
 'Nº Mesa',
 'SubPacto',
 'N°  mesa',
 'Subpacto',
 'MesaNº']

In [6]:
df["Votos TER"] = df["Votos TER"].astype(str).str.strip().replace("", 0).astype(float)

In [7]:
df[df.Candidato.fillna("#").str.contains("NULO")].Candidato.unique()

array(['VOTOS NULOS'], dtype=object)

In [8]:
def create_polling_id(data, columns=[]):
    output = data[columns[0]].fillna("###").copy()

    for column in columns[1:]:
        output += "-" + data[column].fillna("###").astype(str)
    return output

cols = ["Región", "Distrito", "Comuna", "Circunscripción", "Mesa Nº"]
if election_type == "councilors":
    cols += ["Tipo", "Fusionadas"]
df["polling_id"] = create_polling_id(df, columns=cols)

In [9]:
# VOTOS NULOS
# VOTOS EN BLANCO	
# TOTALES
df["Candidato"] = df["Candidato"].replace({
    "VOTOS EN BLANCO": "BLANK",
    "VOTOS EN BLANCOS": "BLANK",
    "VOTOS BLANCOS": "BLANK",
    "VOTOS NULOS": "NULL"
})

df_spoilt = df[df["Candidato"].isin(["NULL", "BLANK"])].copy()
df_spoilt = df_spoilt[["Candidato", "Votos TER", "polling_id"]].rename(columns={"Candidato": "candidate", "Votos TER": "value"})
df_spoilt["flag_candidates"] = 0
df_spoilt.head()

df = df[~df["Candidato"].isin(["BLANK", "NULL", "TOTALES", np.nan])].copy()


In [10]:
data = df[["polling_id", "Candidato", "Partido", "Pacto", "Votos TER"]].dropna(subset=["Pacto"])
data = data.rename(columns={"Pacto": "coalition", "Candidato": "candidate", "Partido": "party", "Votos TER": "value"})
data["value"] = data["value"].astype(str).apply(lambda x: x.strip()).apply(lambda x: 0 if x == "" else x).astype(float)

tt = data.groupby(["polling_id", "coalition", "party", "candidate"]).agg({"value": "sum"})
tt["rate"] = tt.groupby(level=[0], group_keys=False).apply(lambda x: x/x.sum()).fillna(0)
tt = tt.reset_index()
tt["flag_candidates"] = 1

tt.head()

,polling_id,coalition,party,candidate,value,rate,flag_candidates
0,ARICA Y PARINACOTA-1.0-ARICA-ARICA NORTE-###-#...,CHILE DIGNO VERDE Y SOBERANO,FEDERACION REGIONALISTA VERDE SOCIAL,ALEJANDRA ELENA TAPIA ROJAS,0.0,0.000000,1
1,ARICA Y PARINACOTA-1.0-ARICA-ARICA NORTE-###-#...,CHILE DIGNO VERDE Y SOBERANO,FEDERACION REGIONALISTA VERDE SOCIAL,JUAN DOMINGO COÑOMAN QUIÑONES,2.0,0.017241,1
2,ARICA Y PARINACOTA-1.0-ARICA-ARICA NORTE-###-#...,CHILE DIGNO VERDE Y SOBERANO,FEDERACION REGIONALISTA VERDE SOCIAL,LUIS GUILLERMO FIERRO SANDOVAL,2.0,0.017241,1
3,ARICA Y PARINACOTA-1.0-ARICA-ARICA NORTE-###-#...,CHILE DIGNO VERDE Y SOBERANO,FEDERACION REGIONALISTA VERDE SOCIAL,PATRICIA XIMENA SOLANO GARRIDO,2.0,0.017241,1
4,ARICA Y PARINACOTA-1.0-ARICA-ARICA NORTE-###-#...,CHILE DIGNO VERDE Y SOBERANO,INDEPENDIENTES,DAVID JORGE ANTONIO PADILLA RAMIREZ,2.0,0.017241,1


In [11]:
data = pd.concat([
    tt,
    df_spoilt
])

In [12]:
df_location = df[["polling_id", "Región", "Distrito", "Comuna", "Circunscripción", "Local", "Mesa Nº"]].drop_duplicates()
df_location = df_location.rename(columns={
    "Región": "region",
    "Comuna": "commune",
    "Distrito": "district",
    "Circunscripción": "circunscription",
    "Local": "place",
    "Mesa Nº": "polling_station"
})

KeyError: "['Local'] not in index"

In [13]:
data.head()

,polling_id,coalition,party,candidate,value,rate,flag_candidates
0,DE ANTOFAGASTA-DISTRITO 3-ANTOFAGASTA-ANTOFAGA...,CHILE DIGNO VERDE Y SOBERANO,PARTIDO COMUNISTA DE CHILE,PABLO ANDRES IRIARTE RAMIREZ,19.0,0.138686,1
1,DE ANTOFAGASTA-DISTRITO 3-ANTOFAGASTA-ANTOFAGA...,CHILE VAMOS,UNION DEMOCRATA INDEPENDIENTE,ROBERTO CARLOS SOTO ALBALLAY,10.0,0.072993,1
2,DE ANTOFAGASTA-DISTRITO 3-ANTOFAGASTA-ANTOFAGA...,INDEPENDIENTES,INDEPENDIENTES,FABIAN MAURICIO OSSANDON BRICEÑO,18.0,0.131387,1
3,DE ANTOFAGASTA-DISTRITO 3-ANTOFAGASTA-ANTOFAGA...,INDEPENDIENTES,INDEPENDIENTES,JONATHAN RODRIGO VELASQUEZ RAMIREZ,30.0,0.218978,1
4,DE ANTOFAGASTA-DISTRITO 3-ANTOFAGASTA-ANTOFAGA...,INDEPENDIENTES,INDEPENDIENTES,MARIA INES VERDUGO ESCOBAR,13.0,0.094891,1


In [14]:
data.to_csv(f"../../data_output/Chile/2021_{election_type}_polling_station.csv.gz")
df_location.to_csv(f"../../data_output/Chile/2021_{election_type}_polling_station_location.csv.gz")